## Crime Trends in Chicago
#### Cecile Murray

An analysis of crime data from the City of Chicago's open data portal reveals a few key facts about crime trends in Chicago:
* The most common types of crime are theft, battery, and criminal damage. 
* The neighborhoods with the most crime are located on the city's west and south sides. 

In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt

In [61]:
crime17 = pd.read_csv('raw/Crimes-2017.csv')
crime18 = pd.read_csv('raw/Crimes-2018.csv')
comm_areas = gpd.read_file('geo/CHI_comm_areas.geojson')
all_crimes = pd.concat([crime17, crime18])
all_crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11094370,JA440032,09/21/2017 12:15:00 AM,072XX N CALIFORNIA AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,CURRENCY EXCHANGE,True,False,...,50.0,2.0,10,1156443.0,1947707.0,2017,03/01/2018 03:52:35 PM,42.012293,-87.699714,"(42.012293397, -87.699714109)"
1,11118031,JA470589,10/12/2017 07:14:00 PM,055XX W GRAND AVE,1345,CRIMINAL DAMAGE,TO CITY OF CHICAGO PROPERTY,JAIL / LOCK-UP FACILITY,True,False,...,29.0,19.0,14,1138788.0,1913480.0,2017,03/01/2018 03:52:35 PM,41.918712,-87.765511,"(41.918711651, -87.76551063)"
2,11134189,JA491697,10/30/2017 11:52:00 AM,043XX S TALMAN AVE,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,APARTMENT,True,False,...,12.0,58.0,26,1159425.0,1875711.0,2017,03/01/2018 03:52:35 PM,41.814670,-87.690727,"(41.814669784, -87.690726696)"
3,11156462,JA521389,09/29/2017 06:45:00 PM,055XX W BELMONT AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,CURRENCY EXCHANGE,True,False,...,30.0,19.0,11,1138653.0,1920720.0,2017,03/01/2018 03:52:35 PM,41.938581,-87.765831,"(41.938581442, -87.765830579)"
4,11164874,JA531910,12/01/2017 06:20:00 AM,022XX W CHICAGO AVE,0265,CRIM SEXUAL ASSAULT,AGGRAVATED: OTHER,STREET,True,False,...,32.0,24.0,02,1161264.0,1905292.0,2017,03/01/2018 03:52:35 PM,41.895805,-87.683159,"(41.895805099, -87.683159355)"


In [100]:
#comm_areas.head()
comm_xwalk = comm_areas[['community', 'area_num_1']]
comm_xwalk['area_num_1'] = comm_xwalk['area_num_1'].astype(float)

/Users/cecilemurray/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Section 1: Overview 

The table below shows the frequency of each type of crime that had more than 500 occurrances in either 2017 or 2018. Theft, battery, and criminal damage were the most common forms of crime in both 2017 and 2018. While the number of thefts and batteries increased slightly between 2017 and 2018, the number of criminal damage incidents declined by about 4 percent. There were substantial declines in robberies, motor vehicle thefts, and homicides, but notable increases in narcotics offenses and weapons violations.

In [50]:
types17 = crime17[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
types18 = crime18[['Primary Type', 'ID']].groupby(['Primary Type']).count().sort_values(by='ID', ascending=False)
type_summary = types17.join(types18, on='Primary Type', how='outer', lsuffix = '_2017', rsuffix='_2018')
type_summary['Percent Change'] = ((type_summary['ID_2018'] - type_summary['ID_2017']) / type_summary['ID_2017']) * 100
type_summary.columns = ['Number of incidents, 2017', 'Number of incidents, 2018', 'Percent Change']
type_summary.iloc[:19]

,"Number of incidents, 2017","Number of incidents, 2018",Percent Change
Primary Type,,,
THEFT,64345,65079,1.140726
BATTERY,49214,49781,1.152111
CRIMINAL DAMAGE,29042,27806,-4.255905
ASSAULT,19303,20377,5.563902
DECEPTIVE PRACTICE,19025,18716,-1.624179
OTHER OFFENSE,17227,17125,-0.592094
BURGLARY,13001,11729,-9.783863
ROBBERY,11877,9683,-18.472678
NARCOTICS,11658,12987,11.399897


### The most crimes occurred near the Loop and in West Side neighborhoods

The Austin neighborhood experienced the largest number of crimes in 2017 and 2018, followed by River North and the Loop. Several other West Side neighborhoods make the list of the ten neighborhoods with the most crime during this period, as well as two South Side neighborhoods. Meanwhile, the neighborhoods that experienced the smallest number of crimes were largely smaller and farther away from the center of the city.

In [101]:
nhood_summary = all_crimes[['Community Area', 'ID']].groupby(['Community Area']).count().sort_values(by='ID', ascending=False).reset_index()
nhood_summary.rename(columns = {'ID':'Total Crimes', 'Community Area':'area_num_1'}, inplace = True)
nhood_summary = pd.merge(nhood_summary, comm_xwalk, on='area_num_1')
nhood_summary.columns = [['area_num_1', 'Total Crimes', 'Neighborhood']]
nhood_summary[['Neighborhood', 'Total Crimes']].iloc[:10]

,Neighborhood,Total Crimes
0,AUSTIN,30426
1,NEAR NORTH SIDE,25367
2,LOOP,21480
3,NEAR WEST SIDE,18448
4,NORTH LAWNDALE,18376
5,SOUTH SHORE,17325
6,HUMBOLDT PARK,16094
7,WEST TOWN,15625
8,AUBURN GRESHAM,14897
9,WEST ENGLEWOOD,14016


In [102]:
nhood_summary[['Neighborhood', 'Total Crimes']].iloc[-10:-1]

,Neighborhood,Total Crimes
67,BEVERLY,1809
68,FULLER PARK,1716
69,ARCHER HEIGHTS,1666
70,OAKLAND,1344
71,HEGEWISCH,1294
72,MONTCLARE,1206
73,MOUNT GREENWOOD,1149
74,FOREST GLEN,1040
75,BURNSIDE,755


### Theft and battery are the most common crimes in most neighborhoods

In [105]:
nhoods_crime = all_crimes[['Community Area', 'Primary Type', 'ID']].groupby(['Community Area', 'Primary Type']).count().sort_values(by='Community Area').reset_index()
nhoods_crime.columns = ['area_num_1', 'Crime Type', 'Crimes']
nhoods_maxcrime = nhoods_crime.iloc[nhoods_crime.groupby(['area_num_1']).Crimes.idxmax()]
nhoods_maxcrime = pd.merge(nhoods_maxcrime, comm_xwalk, on='area_num_1')
nhoods_maxcrime[[]]


,area_num_1,Crime Type,Crimes,community
0,1.0,THEFT,2282,ROGERS PARK
1,2.0,THEFT,1725,WEST RIDGE
2,3.0,THEFT,1954,UPTOWN
3,4.0,THEFT,1170,LINCOLN SQUARE
4,5.0,THEFT,1057,NORTH CENTER
5,6.0,THEFT,4581,LAKE VIEW
6,7.0,THEFT,4735,LINCOLN PARK
7,8.0,THEFT,12023,NEAR NORTH SIDE
8,9.0,THEFT,115,EDISON PARK
9,10.0,THEFT,571,NORWOOD PARK


Arrest rates vary widely across different types of crime. Nearly all narcotics offenses, concealed carry license violations, and interference with public officer incidents resulted in arrests, while arrests rates were near or below 25 percent for more violent crimes like homicide, assault, battery, and sex offenses. In part, this is because some crimes are only recorded when an arrest occurs; for example, the police can only be aware of a concealed carry license violation if they stop and individual and check them for a concealed weapons. However, the low arrest rates for homicide and other serious violent crimes might still be cause for concern. 

In [14]:
arrests = all_crimes[['Primary Type', 'Arrest', 'ID']].groupby(['Primary Type', 'Arrest']).count().sort_values(by='Primary Type', ascending=False).reset_index()
arrests = arrests.pivot(index='Primary Type', columns='Arrest', values='ID').reset_index()
arrests.columns = ['Primary Type', 'NoArrest', 'Arrest']
arrests['Arrest Rate'] = arrests['Arrest'] / (arrests['Arrest'] + arrests['NoArrest'])
arrests[['Primary Type', 'Arrest Rate']].sort_values(by='Arrest Rate', ascending=False)

,Primary Type,Arrest Rate
17,NARCOTICS,0.999067
4,CONCEALED CARRY LICENSE VIOLATION,0.967890
12,INTERFERENCE WITH PUBLIC OFFICER,0.951903
20,OBSCENITY,0.797688
31,WEAPONS VIOLATION,0.737569
26,PUBLIC PEACE VIOLATION,0.644227
22,OTHER NARCOTIC VIOLATION,0.583333
7,CRIMINAL TRESPASS,0.570064
19,NON-CRIMINAL (SUBJECT SPECIFIED),0.400000
10,HOMICIDE,0.258320


## Section 2: Demographic analysis 

This section describes the demographic characteristics of blocks where certain kinds of crimes commonly occurred. 

### Homicide 

In [106]:
geocrimes = pd.read_csv('temp/merged_crime_acs_data.csv')

In [107]:
geocrimes.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,bg,pct_1parent,pct_alone,pct_white,pct_black,pct_hisp,B15003_001E,pct_nohs,pct_BA,medinc
0,0,11094370,JA440032,09/21/2017 12:15:00 AM,072XX N CALIFORNIA AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,CURRENCY EXCHANGE,True,...,170310202004,0.074257,0.113861,0.378527,0.365451,0.0,1224,NaN,NaN,64019
1,4248,10819643,JA120113,01/17/2017 06:00:00 PM,027XX W TOUHY AVE,0610,BURGLARY,FORCIBLE ENTRY,COMMERCIAL / BUSINESS OFFICE,False,...,170310202004,0.074257,0.113861,0.378527,0.365451,0.0,1224,NaN,NaN,64019
2,13072,10864052,JA170273,02/16/2017 07:00:00 PM,024XX W BIRCHWOOD AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,...,170310202004,0.074257,0.113861,0.378527,0.365451,0.0,1224,NaN,NaN,64019
3,21752,10880042,JA189411,03/16/2017 02:00:00 PM,027XX W CHASE AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,...,170310202004,0.074257,0.113861,0.378527,0.365451,0.0,1224,NaN,NaN,64019
4,22900,10881917,JA191344,03/18/2017 10:00:00 AM,075XX N MAPLEWOOD AVE,0560,ASSAULT,SIMPLE,RESIDENCE,False,...,170310202004,0.074257,0.113861,0.378527,0.365451,0.0,1224,NaN,NaN,64019
